# Book Recommendation System
Content-based Collaborative filtering using
- Author
- Title
- Publisher
- Category

All four point above are the features for the ML Model

<hr>

Make sure to install `opendatasets' on virtual environment, if you dont have just install on notebook with this code cell `!pip install opendatasets`

In [1]:
!pip install opendatasets

In [2]:
import os
import re
import nltk
import requests
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import opendatasets as od
from nltk.corpus import stopwords
nltk.download("stopwords")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from PIL import Image

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
od.download('https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: bintangpratomo
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset


100%|██████████| 76.1M/76.1M [00:01<00:00, 58.7MB/s]


In [4]:
books = pd.read_csv('/content/bookcrossing-dataset/Books Data with Category Language and Summary/Preprocessed_data.csv')
books.head()

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada
3,3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN
4,4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada


In [5]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031175 entries, 0 to 1031174
Data columns (total 19 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Unnamed: 0           1031175 non-null  int64  
 1   user_id              1031175 non-null  int64  
 2   location             1031175 non-null  object 
 3   age                  1031175 non-null  float64
 4   isbn                 1031175 non-null  object 
 5   rating               1031175 non-null  int64  
 6   book_title           1031175 non-null  object 
 7   book_author          1031174 non-null  object 
 8   year_of_publication  1031175 non-null  float64
 9   publisher            1031175 non-null  object 
 10  img_s                1031175 non-null  object 
 11  img_m                1031175 non-null  object 
 12  img_l                1031175 non-null  object 
 13  Summary              1031175 non-null  object 
 14  Language             1031175 non-null  object 
 15

In [6]:
books.isna().sum()

,0
Unnamed: 0,0
user_id,0
location,0
age,0
isbn,0
rating,0
book_title,0
book_author,1
year_of_publication,0
publisher,0


In [7]:
print(books['Category'].unique, '\n')

<bound method Series.unique of 0          ['Social Science']
1               ['Actresses']
2               ['Actresses']
3               ['Actresses']
4               ['Actresses']
                  ...        
1031170             ['Humor']
1031171            ['Nature']
1031172                     9
1031173           ['Fiction']
1031174                     9
Name: Category, Length: 1031175, dtype: object> 



In [8]:
books.Category.value_counts()

,count
Category,
9,406102
['Fiction'],385135
['Juvenile Fiction'],39617
['Biography & Autobiography'],22390
['Humor'],8500
...,...
['Arthur (Fictitious character : Brown)'],1
['Cremation'],1
['Criminal psychology.'],1


In [9]:
print(sorted(books['rating'].unique()), '\n')
books['rating'].value_counts()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 



,count
rating,
0,647323
8,91806
10,71227
7,66404
9,60780
5,45355
6,31689
4,7617
3,5118


In [10]:
books['publisher'].value_counts()

,count
publisher,
Ballantine Books,34724
Pocket,31989
Berkley Publishing Group,28614
Warner Books,25506
Harlequin,25029
...,...
iUniversity Press,1
Drogue Press,1
Broadway Pr,1


<hr>

# Dataset Information
the information gathered from the dataset are
- colum information
    - user_id - id dari pengguna
    - location - lokasi/alamat pengguna
    - age - umur pengguna
    - isbn - kode ISBN (International Standard Book Number) buku
    - rating - rating dari buku
    - book_title - judul buku
    - book_author - penulis buku
    - year_of_publication - tahun terbit buku
    - publisher - penerbit buku
    - img_s - gambar sampul buku (small)
    - img_m - gambar sampul buku (medium)
    - img_l - gambar sampul buku (large)
    - Summary - ringkasan/sinopsis buku
    - Language - bahasa yang digunakan buku
    - Category - kategori buku
    - city - kota pengguna
    - state - negara bagian penguna
    - country - negara pengguna
    
- Null / NaN / Missing Values
    - 1 on book author
    -  14103 on City
    - 22798 on State
    - 35374 on Country
     
- Rating points range are from 0 to 10
- "9" as Category has 406102 on books

<hr>

# Data Preprocessing

- Drop NaN / Null / Missing values
- Drop unused columns
- drop the "9" category
- drop the data that has 0 rating
- Apply the regex for changing the non-alphanumeric with whitespace, then cut the whitespace at front and end of the string


In [11]:
data = books.copy()
data.dropna(inplace=True, how='any', axis=0)
data.reset_index(drop=True, inplace=True)
data.drop(columns = ['Unnamed: 0','location','isbn',
                   'img_s','img_m', 'img_l', 'city','age',
                   'state','Language','country',
                   'year_of_publication', 'Summary'],axis=1,inplace = True)
data.drop(index=data[data.Category == '9'].index, inplace=True)
data.drop(index=data[data.rating == 0].index, inplace=True)
data.Category = data.Category.apply(lambda x: re.sub('[\W_]+', ' ', x).strip())

In [12]:
data.head()

,user_id,rating,book_title,book_author,publisher,Category
1,8,5,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,Actresses
4,67544,8,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,Actresses
7,123629,9,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,Actresses
9,200273,8,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,Actresses
10,210926,9,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,Actresses


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 217314 entries, 1 to 982276
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      217314 non-null  int64 
 1   rating       217314 non-null  int64 
 2   book_title   217314 non-null  object
 3   book_author  217314 non-null  object
 4   publisher    217314 non-null  object
 5   Category     217314 non-null  object
dtypes: int64(2), object(4)
memory usage: 11.6+ MB


In [14]:
data.isnull().sum()

,0
user_id,0
rating,0
book_title,0
book_author,0
publisher,0
Category,0


In [15]:
data['Category'].value_counts()

,count
Category,
Fiction,127055
Juvenile Fiction,14181
Biography Autobiography,8876
Humor,3721
History,3121
...,...
Divers,1
Utah,1
AIDS Disease in children,1


the total category are 4013 categories, that's to much to handle, so if you have PyCharm, it can show first 10 / 20 entries <br>
or you can run the code cell below to show first 25 entries

In [16]:
i = 1
for idx, name in enumerate(data['Category'].value_counts().index.tolist()):
    if i > 25:
        break
    print(i)
    print('Name :', name)
    print('Counts :', data['Category'].value_counts().iloc[idx])

    print('---'*8)
    i+=1

1
Name : Fiction
Counts : 127055
------------------------
2
Name : Juvenile Fiction
Counts : 14181
------------------------
3
Name : Biography Autobiography
Counts : 8876
------------------------
4
Name : Humor
Counts : 3721
------------------------
5
Name : History
Counts : 3121
------------------------
6
Name : Religion
Counts : 2843
------------------------
7
Name : Body Mind Spirit
Counts : 1999
------------------------
8
Name : Juvenile Nonfiction
Counts : 1955
------------------------
9
Name : Social Science
Counts : 1937
------------------------
10
Name : Business Economics
Counts : 1801
------------------------
11
Name : Family Relationships
Counts : 1671
------------------------
12
Name : Self Help
Counts : 1644
------------------------
13
Name : Health Fitness
Counts : 1514
------------------------
14
Name : Cooking
Counts : 1325
------------------------
15
Name : Travel
Counts : 1161
------------------------
16
Name : Poetry
Counts : 985
------------------------
17
Name : Tr

## Picking Category

Pada bagian ini akan dipilih dari category secara spesifik, pemilihan ini dilakukan dikarenakan sumber daya yang dimiliki untuk menghitung tidak memadai untuk menggunakan seluruh category. Category yang akan dipakai adalah 25 category pertama dari data yang sudah di olah

In [17]:
cate_list = data.Category.value_counts().index.tolist()
print(cate_list[:25])
print(len(cate_list[:25]))

['Fiction', 'Juvenile Fiction', 'Biography Autobiography', 'Humor', 'History', 'Religion', 'Body Mind Spirit', 'Juvenile Nonfiction', 'Social Science', 'Business Economics', 'Family Relationships', 'Self Help', 'Health Fitness', 'Cooking', 'Travel', 'Poetry', 'True Crime', 'Psychology', 'Science', 'Computers', 'Literary Criticism', 'Drama', 'Political Science', 'Philosophy', 'Comics Graphic Novels']
25


In [18]:
df = data[data.Category.isin(cate_list[:25])]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183601 entries, 29 to 982276
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      183601 non-null  int64 
 1   rating       183601 non-null  int64 
 2   book_title   183601 non-null  object
 3   book_author  183601 non-null  object
 4   publisher    183601 non-null  object
 5   Category     183601 non-null  object
dtypes: int64(2), object(4)
memory usage: 9.8+ MB


In [19]:
df.Category.nunique()

25

In [20]:
prep = df.copy()
prep.sort_values('book_title')

,user_id,rating,book_title,book_author,publisher,Category
522885,96448,9,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,Hyperion Books for Children,Juvenile Fiction
977437,269557,8,"Ask Lily (Young Women of Faith: Lily Series, ...",Nancy N. Rue,Zonderkidz,Juvenile Fiction
810832,98391,10,Dark Justice,Jack Higgins,Putnam Publishing Group,Fiction
732738,61028,8,Flight of Fancy: American Heiresses (Zebra Ba...,Tracy Cozzens,Kensington Publishing Corporation,Fiction
928726,200323,7,Garfield Bigger and Better (Garfield (Numbere...,Jim Davis,Ballantine Books,Humor
...,...,...,...,...,...,...
447025,260254,7,"Â¿QuÃ© me quieres, amor?",Manuel Rivas,Santillana USA Publishing Company,Fiction
447023,199775,8,"Â¿QuÃ© me quieres, amor?",Manuel Rivas,Santillana USA Publishing Company,Fiction
447022,159274,6,"Â¿QuÃ© me quieres, amor?",Manuel Rivas,Santillana USA Publishing Company,Fiction
447018,12326,6,"Â¿QuÃ© me quieres, amor?",Manuel Rivas,Santillana USA Publishing Company,Fiction


In [21]:
prep = prep.drop_duplicates('book_title')
prep.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55896 entries, 29 to 982276
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      55896 non-null  int64 
 1   rating       55896 non-null  int64 
 2   book_title   55896 non-null  object
 3   book_author  55896 non-null  object
 4   publisher    55896 non-null  object
 5   Category     55896 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.0+ MB


In [22]:
prep.head()

,user_id,rating,book_title,book_author,publisher,Category
29,29526,9,The Kitchen God's Wife,Amy Tan,Putnam Pub Group,Fiction
59,52614,8,What If?: The World's Foremost Military Histor...,Robert Cowley,Berkley Publishing Group,History
66,8,5,Where You'll Find Me: And Other Stories,Ann Beattie,Scribner,Fiction
70,189238,6,Nights Below Station Street,David Adams Richards,Emblem Editions,Fiction
88,8,7,Goodbye to the Buttermilk Sky,Julia Oliver,River City Pub,Fiction


In [23]:
prep['Category'] = prep['Category'].str.replace(' ', '_')
prep.head(10)

,user_id,rating,book_title,book_author,publisher,Category
29,29526,9,The Kitchen God's Wife,Amy Tan,Putnam Pub Group,Fiction
59,52614,8,What If?: The World's Foremost Military Histor...,Robert Cowley,Berkley Publishing Group,History
66,8,5,Where You'll Find Me: And Other Stories,Ann Beattie,Scribner,Fiction
70,189238,6,Nights Below Station Street,David Adams Richards,Emblem Editions,Fiction
88,8,7,Goodbye to the Buttermilk Sky,Julia Oliver,River City Pub,Fiction
94,3329,8,The Testament,John Grisham,Dell,Fiction
501,9,6,Beloved (Plume Contemporary Fiction),Toni Morrison,Plume,Fiction
676,6910,9,Our Dumb Century: The Onion Presents 100 Years...,The Onion,Three Rivers Press,Humor
694,6366,7,New Vegetarian: Bold and Beautiful Recipes for...,Celia Brooks Brown,Ryland Peters & Small Ltd,Cooking
730,14,5,Mary-Kate & Ashley Switching Goals (Mary-Kate ...,Mary-Kate & Ashley Olsen,HarperEntertainment,Juvenile_Fiction


In [24]:
prep.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55896 entries, 29 to 982276
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      55896 non-null  int64 
 1   rating       55896 non-null  int64 
 2   book_title   55896 non-null  object
 3   book_author  55896 non-null  object
 4   publisher    55896 non-null  object
 5   Category     55896 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.0+ MB


In [25]:
book_new = pd.DataFrame({
    'title': prep['book_title'].tolist(),
    'author': prep['book_author'].tolist(),
    'category': prep['Category'].tolist(),
    'publisher': prep['publisher'].tolist()
})
book_new.head()

,title,author,category,publisher
0,The Kitchen God's Wife,Amy Tan,Fiction,Putnam Pub Group
1,What If?: The World's Foremost Military Histor...,Robert Cowley,History,Berkley Publishing Group
2,Where You'll Find Me: And Other Stories,Ann Beattie,Fiction,Scribner
3,Nights Below Station Street,David Adams Richards,Fiction,Emblem Editions
4,Goodbye to the Buttermilk Sky,Julia Oliver,Fiction,River City Pub


# Modeling

## TF-IDF
tahapan ini akan membangun sistem rekomendasi berdasarkan category yang dimiliki buku. Teknik ini digunakan pada sistem rekomendasi untuk menemukan representasi fitur penting dari setiap category buku.

`.fit()` berfungsi untuk melakukan perhitungan terhadap category buku
`.fit_transform()` berguna untuk men-translate hasil perhitungan tersebut kedalam kolom

In [26]:
tfidf = TfidfVectorizer()
tfidf.fit(book_new['category'])
tfidf.get_feature_names_out()

array(['biography_autobiography', 'body_mind_spirit',
       'business_economics', 'comics_graphic_novels', 'computers',
       'cooking', 'drama', 'family_relationships', 'fiction',
       'health_fitness', 'history', 'humor', 'juvenile_fiction',
       'juvenile_nonfiction', 'literary_criticism', 'philosophy',
       'poetry', 'political_science', 'psychology', 'religion', 'science',
       'self_help', 'social_science', 'travel', 'true_crime'],
      dtype=object)

In [27]:
tfidf_matrix = tfidf.fit_transform(book_new['category'])
tfidf_matrix.shape

(55896, 25)

In [28]:
# untuk mendatarkan matrix dan memiliki koneksi ke fitur lain
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

Membuat dataframe untuk melihat tf-idf matrix, Kolom diisi dengan category book, Baris di isi dengan nama book

In [29]:
pd.DataFrame(
    tfidf_matrix.todense(),
    columns=tfidf.get_feature_names_out(),
    index=book_new.title
).sample(5, axis=1).sample(10, axis=0)

,religion,humor,literary_criticism,social_science,drama
title,,,,,
"Brother'S Wife (The Kingsley Baby) (Harlequin Intrigue, No 458)",0.0,0.0,0.0,0.0,0.0
One Small Sparrow,1.0,0.0,0.0,0.0,0.0
My Utmost for His Highest: The Golden Book of Oswald Chambers,1.0,0.0,0.0,0.0,0.0
101 Pet Jokes,0.0,1.0,0.0,0.0,0.0
Guns Save Lives: True Stories of Americans Defending Their Lives With Firearms,0.0,0.0,0.0,0.0,0.0
"Jacob'S Girls (Family Man) (Harlequin Superromance, No 661)",0.0,0.0,0.0,0.0,0.0
"The Mystery in Washington, D.C. (Boxcar Children Special (Paper), No 2)",0.0,0.0,0.0,0.0,0.0
The Dream Society: How the Coming Shift from Information to Imagination Will Transform Your Business,0.0,0.0,0.0,0.0,0.0
How a Seed Grows (Let's-Read-and-Find-Out Science),0.0,0.0,0.0,0.0,0.0


## Cosine Similarity
Menghitung cosine similarity pada matrix tf-idf

In [30]:
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

array([[1., 0., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 1., ..., 0., 0., 1.]])

Make a new dataframe from cosine similarity with row and columns from book_title

In [31]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=book_new['title'], columns=book_new['title'])
print('Shape:', cosine_sim_df.shape)

cosine_sim_df.sample(5, axis=1).sample(3, axis=0)

Shape: (55896, 55896)


title,The Reluctant Suitor,Russia House,"One Palestine, Complete: Jews and Arabs Under the British Mandate",The Time Machine and the Invisible Man,Good Intentions
title,,,,,
Because a Little Bug Went Ka-Choo!,0.0,0.0,0.0,0.0,0.0
LIE TO ME,1.0,1.0,0.0,1.0,1.0
Computers in Your Future (4th Edition),0.0,0.0,0.0,0.0,0.0


In [32]:
def recommendation(book_name, similarity_data=cosine_sim_df, items=book_new[['title', 'category']], k=5):
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,book_name].to_numpy().argpartition(
        range(-1, -k, -1))

    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]

    # Drop nama_resto agar nama resto yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(book_name, errors='ignore')
    df = pd.DataFrame(closest).merge(items)
    df.drop_duplicates(keep='first', subset="title", inplace=True)
    return df.head(k)

In [33]:
book_new.head()

,title,author,category,publisher
0,The Kitchen God's Wife,Amy Tan,Fiction,Putnam Pub Group
1,What If?: The World's Foremost Military Histor...,Robert Cowley,History,Berkley Publishing Group
2,Where You'll Find Me: And Other Stories,Ann Beattie,Fiction,Scribner
3,Nights Below Station Street,David Adams Richards,Fiction,Emblem Editions
4,Goodbye to the Buttermilk Sky,Julia Oliver,Fiction,River City Pub


In [34]:
book_new[book_new['title'].eq("Macromedia Flash MX for Dummies")]

,title,author,category,publisher
42253,Macromedia Flash MX for Dummies,Gurdy Leete,Computers,For Dummies


In [36]:
recommendation("Macromedia Flash MX for Dummies", k=10)

,title,category
0,DocBook: The Definitive Guide (O'Reilly XML),Computers
1,XML in a Nutshell : A Desktop Quick Reference ...,Computers
2,"JavaScript Bible, Gold Edition",Computers
3,Sams Teach Yourself GIMP in 24 Hours,Computers
4,Crypto: How the Code Rebels Beat the Governmen...,Computers
5,"ASP in a Nutshell, 2nd Edition",Computers
6,Palm Computing for Dummies,Computers
7,Programming the Perl DBI,Computers
8,Windows 2000: The Complete Reference,Computers
9,"Using Excel Visual Basic for Applications, Spe...",Computers
